In [36]:
## Configuration
%env riot_api_key='RGAPI-6419882b-a659-434c-a7c3-ec2812172d69'

env: riot_api_key='RGAPI-6419882b-a659-434c-a7c3-ec2812172d69'


In [92]:
## Imports
import pandas as pd
from client import riot_client
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [73]:
# Get all champion ids and create template feature vector
champ_ids = sorted(riot_client.get_all_champions_ids('12.6.1'))
feature_keys = ["t1_{}".format(c_id) for c_id in champ_ids] + ["t2_{}".format(c_id) for c_id in champ_ids] + ["win"]
feature_dict = dict.fromkeys(feature_keys, 0)


In [102]:
# Generate feature vector per match, this will be used as training data.
df = pd.read_csv("client/match_50.csv")[['matchId', 'championId', 'teamId', 'win']]
df.head()
grouped = df.groupby('matchId')


features = []

for m_id, frame in grouped:
    feature_vector = feature_dict.copy()
    t1_won = 0
    for row_index, row in frame.iterrows():
        team = "t{}".format(int(row['teamId']/100))
        champ_team_feature = "{}_{}".format(team, row['championId'])
        feature_vector[champ_team_feature] = 1
        
        current_row_t1_won = 1 if (row['win'] and row['teamId'] == 100) else 0
        t1_won = max(current_row_t1_won, t1_won)
        
    feature_vector['win'] = t1_won
    features.append(feature_vector)

## Training data/independent variables.
feature_df = pd.DataFrame.from_records(features, columns=feature_keys)

# Split dataset into training/test datasets.
X_train, X_test, y_train, y_test = train_test_split(feature_df[feature_df.columns.drop('win')], feature_df.win, test_size=0.2)  

In [103]:
# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [104]:
# Predict and generate classification metrics
y_prediction = model.predict(X_test)
cm = confusion_matrix(y_test, y_prediction)
tp, fp, fn, tn = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
precision, recall = (float(tp/(tp+fp))), float(tp/(tp+fn))
print("TP=[{}], FP=[{}], FN=[{}], TN=[{}]".format(tp, fp, fn, tn))
print("Precision=[{}], Recall=[{}]".format(precision, recall))

TP=[2], FP=[2], FN=[2], TN=[4]
Precision=[0.5], Recall=[0.5]
